In [1]:
from datetime import datetime, timedelta
from pandas import DataFrame
from models.Payload import REGION, RegPayload, AffectedPayload
from models.Source import DataSource as SecApi
from utils.transforms import deltatime_fmt, str2date
from urllib.parse import urlencode as url
#cspell:disable

#input
region:REGION='Valparaiso'
#total regional clients
users = SecApi()
q = users.request('reg_clients',RegPayload(region=region))
clients = int(q[0]['CLIENTES'])
print(f"{"usuarios regionales":<22}",':',f"{clients:,.0f} clientes")

#data
curr = SecApi()
curr.request(source='affected_detail',payload=AffectedPayload()) 
curr_df=curr.dataframe(FECHA_INT_STR=str2date)
print(curr_df.info())
curr_df= curr_df[curr_df['NOMBRE_REGION']==region].drop(columns=['HORA','DIA','MES','ANHO'])
affected = curr_df['CLIENTES_AFECTADOS'].sum()

print(f'{'usuarios sin servicio':<22}',':',f'{affected:,.0f}',' clientes',f'({affected/clients:.2%})')
print(f'{'fecha reporte':<22}',':',curr.report_date.strftime("%d/%m/%Y %H:%M"))

res:DataFrame = curr_df\
    .groupby(['NOMBRE_COMUNA','NOMBRE_EMPRESA','FECHA_INT_STR'],as_index=False)['CLIENTES_AFECTADOS']\
    .sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)
    
res['TIEMPO_SIN_SERVICIO'] = res['FECHA_INT_STR'].apply(lambda event_date: datetime.now()-event_date+timedelta(days=0))

#cspell:disable
res:DataFrame = res.rename(columns={
    'FECHA_INT_STR':'INCIDENTE',
    'CLIENTES_AFECTADOS':'SIN_SERVICIO',
    'NOMBRE_EMPRESA':'DISTRIBUIDORA',
    'NOMBRE_COMUNA':'COMUNA'
    })

#days w/o service
res['TIEMPO_SIN_SERVICIO'] = res['INCIDENTE'].apply(deltatime_fmt)
res

usuarios regionales    : 780,563 clientes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687 entries, 0 to 686
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DIA_INT             687 non-null    int64         
 1   MES_INT             687 non-null    int64         
 2   ANHO_INT            687 non-null    int64         
 3   HORA                687 non-null    int64         
 4   DIA                 687 non-null    int64         
 5   MES                 687 non-null    int64         
 6   ANHO                687 non-null    int64         
 7   NOMBRE_REGION       687 non-null    object        
 8   NOMBRE_COMUNA       687 non-null    object        
 9   NOMBRE_EMPRESA      687 non-null    object        
 10  CLIENTES_AFECTADOS  687 non-null    int64         
 11  ACTUALIZADO_HACE    687 non-null    object        
 12  FECHA_INT_STR       687 non-null    datetime64[ns]
dtypes: datet

,COMUNA,DISTRIBUIDORA,INCIDENTE,SIN_SERVICIO,TIEMPO_SIN_SERVICIO
9,Viña del Mar,CHILQUINTA,2024-02-19,155,190 d 17 h
5,Petorca,CGE,2024-08-27,96,0 d 17 h
1,El Quisco,LITORAL,2024-08-27,85,0 d 17 h
6,Quilpue,CHILQUINTA,2024-02-19,37,190 d 17 h
8,Viña del Mar,CGE,2024-08-27,4,0 d 17 h
2,El Tabo,LITORAL,2024-08-27,2,0 d 17 h
0,Cabildo,CGE,2024-08-27,1,0 d 17 h
3,La Ligua,CGE,2024-08-27,1,0 d 17 h
4,Papudo,CGE,2024-08-27,1,0 d 17 h
7,Santo Domingo,CGE,2024-08-27,1,0 d 17 h


In [2]:
res_by_city:DataFrame = curr_df.groupby(['NOMBRE_COMUNA'],as_index=False)['CLIENTES_AFECTADOS']\
    .sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)
res_by_city

,NOMBRE_COMUNA,CLIENTES_AFECTADOS
8,Viña del Mar,159
5,Petorca,96
1,El Quisco,85
6,Quilpue,37
2,El Tabo,2
0,Cabildo,1
3,La Ligua,1
4,Papudo,1
7,Santo Domingo,1


In [3]:
from models.Payload import Payload,AffectedPayload
from models.Source import DataSource as Api
from utils.transforms import str2date,deltatime_fmt
from pandas import DataFrame
#cspell:disable
api = Api()
delay_folder = [{'delay_hours':0},{'delay_hours':1},{'delay_hours':6},{'delay_hours':12},{'delay_hours':24}]
data_folder = []
for delay in delay_folder:
    api.request(source='affected_detail',payload=AffectedPayload(**delay))
    result:DataFrame = api.dataframe(FECHA_INT_STR=str2date).copy()
    ## transform
    ### rename
    result = result.rename(columns={
        "DIA_INT":"día",
        "MES_INT":"mes",
        "ANHO_INT":"año",
        "NOMBRE_REGION":"región",
        "NOMBRE_COMUNA":"comuna",
        "NOMBRE_EMPRESA":"distribuidora",
        "ACTUALIZADO_HACE":"actualización",
        "FECHA_INT_STR":"fecha incidente",
        "CLIENTES_AFECTADOS":"afectados",
        })

    ###calculate columns
    result['periodo sin servicio']=result['fecha incidente'].apply(deltatime_fmt)

    ###remove
    result = result.drop(columns=['HORA','DIA','MES','ANHO'])
    
    ###append
    data_folder.append(result)

print(data_folder)

[     día  mes   año         región      comuna distribuidora  afectados  \
0     27    8  2024   La Araucania      Lumaco       FRONTEL          1   
1     27    8  2024       Los Rios    Valdivia         SAESA          1   
2     27    8  2024          Maule      Retiro     LUZPARRAL          1   
3     27    8  2024     Valparaiso      Papudo           CGE          1   
4     27    8  2024   La Araucania      Temuco           CGE          1   
..   ...  ...   ...            ...         ...           ...        ...   
682   27    8  2024      Los Lagos     Queilen         SAESA          1   
683   27    8  2024  Metropolitana    La Reina          ENEL          1   
684   27    8  2024  Metropolitana  Las Condes          ENEL          1   
685   27    8  2024          Maule      Molina           CGE         86   
686   27    8  2024   La Araucania       Pucon           CGE          1   

                  actualización fecha incidente periodo sin servicio  
0    0 Dias 0 Horas 14 Minu